# Tools with MCP ⏰

The Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `langchain-mcp-adapters`.

## Setup

Load and/or check for needed environmental variables

In [3]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OLLAMA_HOST_URL=http://localhost:11434


In [4]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio
import os

nest_asyncio.apply()

math_server_path = "./servers/math_server.py"

mcp_client = MultiServerMCPClient(
    {
            "math": {
                "command": "python3",
                "args": [math_server_path],
                "transport": "stdio",
            }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

Loaded 2 MCP tools: ['add', 'multiply']


Create an agent with the MCP-provided time tools.

In [5]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
import os

model = ChatOllama(
    model="granite4:latest",
    temperature=0,
    base_url=os.environ['OLLAMA_HOST_URL']
)

agent_with_mcp = create_agent(
    model=model,
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

Ask about the current time in San Francisco.

In [6]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's 4 * 5?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What's 4 * 5?
================================== Ai Message ==================================
Tool Calls:
  multiply (72c3e936-eee6-4f56-be9f-8832696415e5)
 Call ID: 72c3e936-eee6-4f56-be9f-8832696415e5
  Args:
    a: 4
    b: 5
================================= Tool Message =================================
Name: multiply

20
================================== Ai Message ==================================

The result of multiplying 4 by 5 is **20**.
